## Import Library

In [122]:
from google.colab import drive
drive.mount('/content/drive')
project_path = "/content/drive/MyDrive/ComBio"
!pip install imbalanced-learn scikit-learn matplotlib seaborn pandas

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [123]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler

## Import Dataset

In [124]:
file_path = f"{project_path}/Dataset/Processed/raw_dataset.csv"
result_path = f"{project_path}/Result"
hyper_path = f"{project_path}/Hyperparameter"
model_path = f"{project_path}/Models"

df = pd.read_csv(file_path)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47939 entries, 0 to 47938
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  47939 non-null  float64
 1   gender               47939 non-null  float64
 2   race_ethnicity       47939 non-null  float64
 3   education_level      38185 non-null  float64
 4   income_ratio         43219 non-null  float64
 5   marital_status       39842 non-null  float64
 6   jaundice_history     20076 non-null  float64
 7   receive_transfusion  47939 non-null  float64
 8   liver_condition      38184 non-null  float64
 9   injectable_drug_use  28684 non-null  float64
 10  drug_use             28691 non-null  float64
 11  alcohol_consumption  30525 non-null  float64
 12  alt                  44261 non-null  float64
 13  albumin              44361 non-null  float64
 14  alp                  44352 non-null  float64
 15  ast                  44237 non-null 

## Cleaning

In [125]:
df['age'] = df['age'].astype('Int64')
df['income_ratio'] = df['income_ratio'].astype('float')

df['gender'] = df['gender'].astype('Int64')
df['race_ethnicity'] = df['race_ethnicity'].astype('Int64')

df['education_level'] = df['education_level'].replace({7:1, 8: 1, 9:1}).astype('Int64')
df['marital_status'] = df['marital_status'].replace({77: 5, 99: 5}).astype('Int64')
df['jaundice_history'] = df['jaundice_history'].replace({9: 2}).astype('Int64')

df['receive_transfusion'] = df['receive_transfusion'].replace({7: 2, 9: 2}).astype('Int64')
df['liver_condition'] = df['liver_condition'].replace({9: 2}).astype('Int64')
df['injectable_drug_use'] = df['injectable_drug_use'].replace({7: 2, 9: 2}).astype('Int64')
df['drug_use'] = df['drug_use'].replace({7: 2, 9: 2}).astype('Int64')
df['alcohol_consumption'] = df['alcohol_consumption'].replace({9: 2}).astype('Int64')

blood_test_columns = [
    "alt", "albumin", "alp", "ast", "bun", "cholesterol", "cpk", "creatinine",
    "ggt", "glucose", "total_bilirubin", "total_calcium", "total_protein",
    "triglycerides", "uric_acid", "ldh", "globulin", "osmolality"
]
df[blood_test_columns] = df[blood_test_columns].astype('float')

df['hcv_status'] = df['hcv_status'].replace({2: 0, 3: 0}).astype('Int64')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47939 entries, 0 to 47938
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  47939 non-null  Int64  
 1   gender               47939 non-null  Int64  
 2   race_ethnicity       47939 non-null  Int64  
 3   education_level      38185 non-null  Int64  
 4   income_ratio         43219 non-null  float64
 5   marital_status       39842 non-null  Int64  
 6   jaundice_history     20076 non-null  Int64  
 7   receive_transfusion  47939 non-null  Int64  
 8   liver_condition      38184 non-null  Int64  
 9   injectable_drug_use  28684 non-null  Int64  
 10  drug_use             28691 non-null  Int64  
 11  alcohol_consumption  30525 non-null  Int64  
 12  alt                  44261 non-null  float64
 13  albumin              44361 non-null  float64
 14  alp                  44352 non-null  float64
 15  ast                  44237 non-null 

In [126]:
df.isnull().sum()

,0
age,0
gender,0
race_ethnicity,0
education_level,9754
income_ratio,4720
marital_status,8097
jaundice_history,27863
receive_transfusion,0
liver_condition,9755
injectable_drug_use,19255


In [127]:
df = df[~df['hcv_status'].isna()]

In [ ]:
df_hcv2 = df[df['hcv_status'] == 0].copy() # Negative
df_hcv2.dropna(inplace=True)
df_hcv2.drop_duplicates(inplace=True)

df_hcv1 = df[df['hcv_status'] == 1].copy() # Positive
df_hcv1.fillna(df_hcv1.median(numeric_only=True), inplace=True)

df = pd.concat([df_hcv2, df_hcv1], ignore_index=True)

In [129]:
df.isnull().sum()

,0
age,0
gender,0
race_ethnicity,0
education_level,0
income_ratio,0
marital_status,0
jaundice_history,0
receive_transfusion,0
liver_condition,0
injectable_drug_use,0


In [130]:
print(df['hcv_status'].value_counts())

hcv_status
0    6903
1     441
Name: count, dtype: Int64


In [ ]:
#  Ambil 2000 data negative saja
hcv2 = df[df['hcv_status'] == 0]
hcv2_sampled = hcv2.sample(n=2000, random_state=42)

df = pd.concat([
    hcv2_sampled,
    df[df['hcv_status'] != 0]
], ignore_index=True)

In [132]:
int64_columns = df.select_dtypes(include=['Int64']).columns

for col in int64_columns:
    df[col] = df[col].astype('int')

In [133]:
X = df.drop(columns=['hcv_status'])
y = df['hcv_status']


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

smote = SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

train = pd.DataFrame(X_train_smote, columns=X_train.columns)
train['hcv_status'] = y_train_smote
train['dataset'] = 'train'

val = X_val.copy()
val['hcv_status'] = y_val
val['dataset'] = 'validation'

test = X_test.copy()
test['hcv_status'] = y_test
test['dataset'] = 'test'

df_cleaned = pd.concat([train, val, test], axis=0).reset_index(drop=True)

In [134]:
# Total keseluruhan
print("== Total Keseluruhan ==")
print(df_cleaned['hcv_status'].value_counts())
print()

# Train set
print("== Train Set ==")
print(df_cleaned[df_cleaned['dataset'] == 'train']['hcv_status'].value_counts())
print()

# Validation set
print("== Validation Set ==")
print(df_cleaned[df_cleaned['dataset'] == 'validation']['hcv_status'].value_counts())
print()

# Test set
print("== Test Set ==")
print(df_cleaned[df_cleaned['dataset'] == 'test']['hcv_status'].value_counts())

== Total Keseluruhan ==
hcv_status
0    2000
1    1687
Name: count, dtype: int64

== Train Set ==
hcv_status
0    1599
1    1599
Name: count, dtype: int64

== Validation Set ==
hcv_status
0    200
1     44
Name: count, dtype: int64

== Test Set ==
hcv_status
0    201
1     44
Name: count, dtype: int64


In [135]:
df_cleaned.to_csv(f'{project_path}/Dataset/cleaned_dataset.csv', index=False)